In [1]:
%matplotlib inline
import os
import csv
import fnmatch
import numpy as np
import operator
import re
from pathlib import Path

In [73]:
def FixScratchPad(ScratchPad_Old):
    # Fix the situation where comma has divided decimals into separate columns
    
    ScratchPad_Times = [item[0] for item in ScratchPad_Old]
    
    ScratchPad_Phrases = list()
    
    # loop to combine phrases divided by commas
    ScratchPadInd = -1 
    while ScratchPadInd < len(ScratchPad_Old)-1:
        ScratchPadInd = ScratchPadInd + 1
        commasInPhrase = len(ScratchPad_Old[ScratchPadInd])-2
        if commasInPhrase < 1:
            #print(ScratchPad_Old[ScratchPadInd][1])
            ScratchPad_Phrases.append(ScratchPad_Old[ScratchPadInd][1])
            continue
        scratchPadPhrase = ScratchPad_Old[ScratchPadInd][1]
        for phraseJoinNr in range(1, commasInPhrase+1):
            scratchPadPhrase = scratchPadPhrase + ', ' + ScratchPad_Old[ScratchPadInd][1+phraseJoinNr]
        
        ScratchPad_Phrases.append(scratchPadPhrase)
            
        
    ScratchPad_New = [[ScratchPad_Times[ind], ScratchPad_Phrases[ind]] for ind in 
                    range(0, len(ScratchPad_Times))]
    
    #UserKeys_New = np.concatenate((UserKeys_Times, UserKeys_Keys, UserKeys_ProgressPercent), axis = 0)
    
    #print(ScratchPad_New)
    return ScratchPad_New

In [161]:
def stimPhrasesEdit(PhraseLog, subjName):
    
    # Now extract phrases from the phrase file
    phraseStim_Phrases = [item[1] for item in PhraseLog]
    phraseStim_PhrasesReduced = sorted(set(phraseStim_Phrases), key=phraseStim_Phrases.index)
    
    PhraseLogReduced= list()
    ind = -1
    
    for i in phraseStim_PhrasesReduced:
        ind = ind + 1

        if ind == len(phraseStim_PhrasesReduced)-1:
            if subjName == 'sa\Test_woChinRest\p1':
                PhraseLogReduced.append([phraseLog[phraseStim_Phrases.index(i)][0], i[0:31]])
                
            elif subjName == 'rh\Test_wChinRest\p1':
                PhraseLogReduced.append([phraseLog[phraseStim_Phrases.index(i)][0], i[0:-9]])
            else:
                PhraseLogReduced.append([phraseLog[phraseStim_Phrases.index(i)][0], i])
        else:
            PhraseLogReduced.append([phraseLog[phraseStim_Phrases.index(i)][0], i])
            
    del PhraseLogReduced[0]
    del PhraseLogReduced[0]
    

    
    if PhraseLogReduced[-1][1] == 'THE EXPERIMENT IS NOW DONE':
        del PhraseLogReduced[-1]
        
    if subjName == 'rh\Test_wChinRest\p2':
        #print(PhraseLogReduced[2])
        del PhraseLogReduced[2]
        #print(PhraseLogReduced[-1])
        del PhraseLogReduced[-1]
        
    #if subjName == 'sa\Test_woChinRest\p1' or subjName == 'rh\Test_wChinRest\p1':
    #    del PhraseLogReduced[-1]
        
    if subjName == 'sa\Test_woChinRest\p2':
        del PhraseLogReduced[3]
        del PhraseLogReduced[1]
        del PhraseLogReduced[0]
        
    return PhraseLogReduced
    
    

In [162]:
def scratchPadPhraseEdit(phraseUser, subjName):
    phraseUserEnd = list()
    
    
    #print(phraseUser)
    for row_ind in range(0, len(phraseUser)):
        if row_ind!= 0 and phraseUser[row_ind][1] == '':
            if len(phraseUser[row_ind-1][1])>10:
                phraseUserEnd.append(phraseUser[row_ind-1])
                #print(phraseUser[row_ind-1])
    
    if subjName == 'sa\Test_woChinRest\p1':
        phraseUserEnd.append([phraseUser[-1][0], phraseUser[-1][1][0:30]])
    
    if subjName == 'rh\Test_wChinRest\p1':
        phraseUserEnd.append(phraseUser[-1])
        
    # remove first trial of text composition
    del phraseUserEnd[0]
        
    if subjName == 'rh\Test_wChinRest\p2':
        #print(phraseUserEnd[-2])
        del phraseUserEnd[-2]
        
    return phraseUserEnd

In [163]:
# find minimum cost and the operations that give rise to it
def minValnInd(costOptions, flagSame):
    operator = list()
    unique_entries = set(costOptions)
    valInd = { value : [ i for i, v in enumerate(costOptions) if v == value ] for value in unique_entries }
    keyVal = list(valInd.keys())
    min_value = min(keyVal)
    
    if 0 in valInd[min_value]:
        operator.append('D')
    if 1 in valInd[min_value]:
        operator.append('I')
    if 2 in valInd[min_value]:
        if flagSame == 0:
            operator.append('S')
        else:
            operator.append('N')   
    flagSame = None    
    return min_value, ''.join(operator)

In [164]:
# Function to compute the MSD, with cost of 2 for substitution and 1 for insertion and deletion
costSub = 2
costIns = 1
costDel = 1

def levenshteinDist(phraseIn, phraseOut):
    
    lenStim = len(phraseIn)
    lenUser = len(phraseOut)
    costMatrix = np.zeros((lenStim+1, lenUser+1), dtype=int)
    MSDoperation = np.empty([lenStim+1, lenUser+1], dtype="U4")
    costMatrix[0,0:] = range(0, lenUser+1)
    costMatrix[0:,0] = range(0, lenStim+1)
    MSDoperation[0,0:] = 'I'
    MSDoperation[0:,0] = 'D'
    
    for i in range(1, len(phraseIn)+1):
        iP = i - 1
        for j in range(1, len(phraseOut)+1):
            jP = j - 1
            if phraseIn[iP].lower() == phraseOut[jP].lower():
                # Define the possible cost array
                costOptionArray = [costMatrix[i,j-1]+costDel, costMatrix[i-1,j]+costIns, costMatrix[i-1,j-1]] 
                flagSame = 1
            else:
                costOptionArray = [costMatrix[i,j-1]+costDel, costMatrix[i-1,j]+costIns, costMatrix[i-1,j-1]+costSub]
                flagSame = 0
            costMatrix[i,j], MSDoperation[i][j] = minValnInd(costOptionArray, flagSame)
    #print(costMatrix)
    return costMatrix[-1,-1]


In [166]:
metricComputed = 'error_rate'
dataFolderName = r'C:\DTU\Data\201812_ExptToCheckMovementEffect\Data'

errorRateList = list()
TypingSpeed = list()
Names = list()

for root, dirs, subfolder in os.walk(dataFolderName):
    
    if not dirs and 'rh\Test_wChinRest' in root:
        
        #if 'notCompleted' in root or 'notInclude' in root: # Some subjects do not have gaze log and have been marked as 
        
        if 'notCompleted' in root: # Some subjects do not have gaze log and have been marked as 
            #notInclude
            continue
        if 'tb' in root or 'trial' in root or 'he\Test_woChinRest' in root:
            continue
            
        scratchPad = None
        phraseLog = None
        
        for file in subfolder:
            
            if fnmatch.fnmatch(file, 'ScratchPadLog*'):
                try:
                    fScratchPad = open(root + '\\' + file, encoding='utf-8')
                    readerScratchPad = csv.reader(fScratchPad)
                    scratchPad = list(readerScratchPad)
                except:
                    if fScratchPad is not None:
                        fScratchPad.close()
                    else:
                        print('error in opening the scratchpad log file')
                        
            if fnmatch.fnmatch(file, 'phrase*'):
                try:
                    fPhraseLog = open(root + '\\' + file, encoding='utf-8')
                    readerPhraseLog = csv.reader(fPhraseLog)
                    phraseLog = list(readerPhraseLog)    
                        
                except:
                    if fPhraseLog is not None:
                        print('closing')
                        fPhraseLog.close()
                    else:
                        print('error in opening the phrase log file')
            
            if fnmatch.fnmatch(file, 'ReplacementPhraseLog.csv'):
                
                try:
                    fPhraseLog = open(root + '\\' + file)
                    readerPhraseLog = csv.reader(fPhraseLog)
                    phraseLog = list(readerPhraseLog)
                        
                except:
                    if fPhraseLog is not None:
                        print('closing')
                        fPhraseLog.close()
                    else:
                        print('error in opening the replacement phrase log file')
                     
        if scratchPad is None or phraseLog is None:
            #print(root)
            continue
        else:
            a = re.compile('(?<=ExptToCheckMovementEffect\\\\Data\\\\)(.*)(?=\\\\2018-1)')
            subjName = a.findall(root)[0]
            print(subjName)
            
            if 'sa\Test_woChinRest\p1' in subjName or subjName == 'rh\Test_wChinRest\p1':
                scratchPad = scratchPad[:-1]
            
             # fix userKeys due to comma related file changes
            scratchPad_new = FixScratchPad(scratchPad)
            
            #print(scratchPad_new)
            # fix userKeys due to comma related file changes
            phraseLog_new = FixScratchPad(phraseLog)
            
            phraseLog_reduced = stimPhrasesEdit(phraseLog_new, subjName)
            
            phraseUserEnd = scratchPadPhraseEdit(scratchPad_new, subjName)
            
            print( len(phraseLog_reduced), len(phraseUserEnd))
                        
            if 'rh\Test_wChinRest\p2' in subjName or 'sa\Test_woChinRest\p2' in subjName:
                "Do not modify distance"
            else:
                dist = 0
            
            #print(phraseLog_reduced)
            #print(phraseUserEnd)

            if len(phraseUserEnd)==len(phraseLog_reduced):
                for n in range(0,len(phraseLog_reduced)):
                    #print(phraseLog_reduced[n][1], phraseUserEnd[n][1])
                    dist = dist + (levenshteinDist(phraseLog_reduced[n][1], phraseUserEnd[n][1]))/max(len(phraseLog_reduced[n][1]),len(phraseUserEnd[n][1]))
            else:
                print('Unequal stimulation and user-generated phrases')

            if 'rh\Test_wChinRest' in subjName or 'sa\Test_woChinRest' in subjName:
                if 'p1' in subjName:
                    dist1 = dist
                    continue
                else:
                    dist2 = dist
                    dist = dist1 + dist2
            
#             if 'sa\Test_woChinRest\p2' in subjName or 'rh\Test_wChinRest\p2' in subjName:
#                 errorRate = dist/8
#                 print('LESS TRIALS')
#             else:
#                 errorRate = dist/9 # All subjects, except 2 in this experiment have 9 trials per session
            errorRate = dist/9
            errorRateList.append(errorRate*100) # save error rate in percentage
            print('Minimum edit distance by the user is ', errorRate)
                
            
            
#if errorRateList:
#    #print(Names)
#    AddDataToFile(metricComputed, Names, errorRateList)
            

rh\Test_wChinRest\p1
2 2
rh\Test_wChinRest\p2
7 7
Minimum edit distance by the user is  0.10911438251362922


In [12]:
def AddDataToFile(metricComputed, Names, DataEvaluated):

    # Save the data
    fileIn = r'C:\DTU\Data\201805_HealthnRehab\data_summary.csv'

    if Path(fileIn).is_file():
        print(fileIn)
        fSize = os.path.getsize(fileIn) # or pd.read_excel(filename) for xls file

        # file exists
        # check if the given column exists
        with open(fileIn, 'r', newline='') as csvfileRead:
            if fSize > 0:
                fileOut = fileIn[:-4] + '1.csv'
                with open(fileOut, 'w', newline='') as csvfileWrite:
                    for row in csv.reader(csvfileRead):
                        if metricComputed in row:
                            # file exists and the metric is present in the file
                            print('This metric is already calculated and saved: ', metricComputed)
                            break
                        else:
                            # file exists but the metric is not present
                            print('adding the metric')
                            filewriter = csv.writer(csvfileWrite, delimiter = ',')
                            if len(row) > 1:
                                rowJoined = list()
                                for rowElement in row:
                                    rowJoined.append(rowElement)
                                
                                
                                print(rowJoined[1])
                                
                                if rowJoined[1] == 'subject_name':
                                    # write the titles of the metrics
                                    rowJoined.append(metricComputed)
                                    filewriter.writerow(rowJoined)
                                else:
                                    #filewriter.writerow([row, str(DataEvaluated[Names.index(row[0])])])
                                    rowJoined.append(str(DataEvaluated[Names.index(row[1])]))
                                    filewriter.writerow(rowJoined)
                                
                            else:
                                if row[1] == 'subject_name':
                                    # write the titles of the metrics
                                    filewriter.writerow([row, metricComputed])
                                else:
                                    filewriter.writerow([row, str(DataEvaluated[Names.index(row[1])])])
            
            
            else:
                fileOut = fileIn
                print('recreating a file')

                with open(fileSave, 'w', newline='') as csvfileWrite:
                    filewriter = csv.writer(csvfileWrite, delimiter=',')
                    filewriter.writerow(['subject_name', metricComputed])
                    for ind in enumerate(Names):
                        filewriter.writerow([Names[ind], DataEvaluated[ind[0]]])
                
    else:
        print('creating a file')
        a = re.compile('(?<=TypingData)(.*)(?=OptiKey)')
        with open(fileSave, 'w', newline='') as csvfileWrite:
            filewriter = csv.writer(csvfileWrite, delimiter=',')
            filewriter.writerow(['subject_name', metricComputed])
            for ind in enumerate(Names):
                subjName = a.findall(Names[ind[0]])
                filewriter.writerow([subjName[0], DataEvaluated[ind[0]]])


In [ ]:
subjName = r'C:\DTU\Data\201805_HealthnRehab\TypingData\May16\pt_MS'


for root, dirs, subfolder in os.walk(subjName):
    if not dirs:
        if 'notCompleted' in root:
            continue
        for file in subfolder:
            if fnmatch.fnmatch(file, 'PhraseLog*'):
                with open(root + '\\' + file, encoding='utf-8') as fp:
                    reader = csv.reader(fp)
                    phraseStim = list(reader)
                    phraseStim.remove(phraseStim[0])
            elif fnmatch.fnmatch(file, 'ScratchPadLog*'):
                with open(root + '\\' + file, encoding='utf-8') as fs:
                    reader = csv.reader(fs)
                    phraseUser = list(reader)
                    phraseUser.remove(phraseUser[0])      

In [ ]:
phraseUserEnd = list()

for row_ind in range(0, len(phraseUser)):
    if row_ind!= 0 and phraseUser[row_ind][1] == '':
        phraseUserEnd.append(phraseUser[row_ind-1])

In [ ]:
# Attempt to remove all elements that are not end phrases - stimulated or user-generated

for row_ind2 in range(0, len(phraseStim)):
    if phraseStim[row_ind2][1] == 'THE EXPERIMENT IS NOW DONE':
        phraseUserEnd.remove(phraseUserEnd[row_ind2])
        phraseStim.remove(phraseStim[row_ind2])
        
for row_ind3 in range(0, len(phraseUserEnd)):
    if phraseUserEnd[row_ind3][1] == '':
        if row_ind3 < len(phraseStim):
            phraseStim.remove(phraseStim[row_ind3])
            phraseUserEnd.remove(phraseUserEnd[row_ind3])
    
# if phrases documented have a comma, the said phrase is split and saved into different columns. They need to be joined
for row_ind4 in range(0, len(phraseStim)):
    if len(phraseStim[row_ind4])>2:
        phraseStim[row_ind4][1] = ''.join(phraseStim[row_ind4][1:])
        
for row_ind5 in range(0, len(phraseUserEnd)):
    if len(phraseUserEnd[row_ind5])>2:
        phraseUserEnd[row_ind5][1] = ''.join(phraseUserEnd[row_ind5][1:])
        


In [ ]:
dist = 0
if len(phraseUserEnd)==len(phraseStim):
    for n in range(0,len(phraseStim)):
        dist = dist + (levenshteinDist(phraseStim[n][1], phraseUserEnd[n][1]))/max(len(phraseStim[n][1]),len(phraseUserEnd[n][1]))
        print(levenshteinDist(phraseStim[n][1], phraseUserEnd[n][1]), max(len(phraseStim[n][1]),len(phraseUserEnd[n][1])), n)
    dist = float(dist/(n+1))
    print("Minimum edit distance by the user is ", dist)
else:
    print('Unequal stimulation and user-generated phrases')


In [ ]:
phraseStim

In [ ]:
phraseUserEnd